In [ ]:
import pandas as pd
import numpy as np
import torch
import re
import tqdm
import xgboost as xgb

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Masking
from tensorflow.keras.optimizers import Adam

from sklearn.utils import shuffle
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

from sklearn.metrics import matthews_corrcoef,accuracy_score, confusion_matrix, classification_report, f1_score
from sklearn.model_selection import train_test_split

from tqdm import tqdm
from tqdm.keras import TqdmCallback
from transformers import AutoTokenizer, AutoModel, RobertaModel, RobertaTokenizer

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import tensorflow as tf
print("GPU Available:", tf.config.list_physical_devices('GPU'))

GPU Available: []


# Read Dataset

In [ ]:
# embedding = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Gammafest 2025/specter_embedding.csv')
metadata = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Gammafest 2025/combined_with_nan.csv')

mtdt = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Gammafest 2025/mtdt_embedding.csv')
txt = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Gammafest 2025/txt_embedding.csv')

train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Gammafest 2025/train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Gammafest 2025/test.csv')
sample = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Gammafest 2025/sample_submission.csv')

Preparation

In [ ]:
txt_embed = txt.drop(columns=['Unnamed: 0'])
mtdt_embed = mtdt.drop(columns=['Unnamed: 0', 'paper_id'])

In [ ]:
data_embed = pd.concat([txt_embed, mtdt_embed], axis=1)
data_embed

,paper_id,dim_0,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6,dim_7,dim_8,...,mtdt_758,mtdt_759,mtdt_760,mtdt_761,mtdt_762,mtdt_763,mtdt_764,mtdt_765,mtdt_766,mtdt_767
0,p0000,-0.307896,0.233469,-0.795748,-0.020367,-0.135363,0.108508,1.538086,1.053744,0.992476,...,0.361256,0.309300,0.940870,0.438250,-0.097464,-0.177169,0.318410,0.378203,-0.130136,0.547858
1,p0001,0.084872,0.392815,-0.504959,-0.021132,-0.114442,0.422108,0.588093,0.723649,0.348245,...,0.345426,-0.708377,0.869000,0.223402,-0.416547,0.202308,-0.492604,0.643108,-0.323697,0.918231
2,p0002,-0.028009,0.680236,-0.821320,0.205567,0.157409,1.311699,-0.855878,0.208994,1.789679,...,-0.080286,-0.225180,0.869219,0.469897,0.418345,0.023076,0.550739,0.508122,-0.765913,1.141435
3,p0003,-0.462517,1.123400,0.482200,0.050586,0.252007,-0.154191,0.874468,1.153361,0.787142,...,0.327962,-1.480308,0.990188,1.004630,-1.131652,0.046745,0.652956,0.394007,0.403567,0.913617
4,p0004,-0.686830,1.164063,0.139771,-0.061362,-0.311246,0.477096,1.062190,0.358880,0.112960,...,0.267060,-0.313841,0.427698,0.400070,-0.242398,0.235142,0.353829,0.140394,0.488866,0.461436
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4349,p4349,-0.228524,0.531688,-0.480547,0.038501,-0.109922,-0.126173,0.954068,0.801017,0.071041,...,-0.106525,-1.055142,0.591994,-0.062940,-0.504965,0.337807,0.331830,0.070886,0.197972,0.950505
4350,p4350,-0.760740,0.784616,-0.522071,-0.182086,-0.432741,0.345590,-0.227202,0.635391,0.787055,...,0.678842,-0.207159,1.074868,0.143668,-0.590506,0.170782,0.200454,-0.447116,-0.931619,0.611930
4351,p4351,0.003420,0.617399,-0.119199,-0.603085,0.375625,-0.284949,0.108157,-0.025868,0.097006,...,0.406650,0.062788,0.820162,0.283859,-0.373997,-0.174518,0.225618,-0.155076,-0.131684,0.226827
4352,p4352,-0.822705,0.531300,0.687420,-0.998790,0.647437,-0.258114,0.966562,0.221419,0.495811,...,0.824986,-0.652105,0.342011,1.322424,-0.120471,0.523878,0.667718,0.203245,0.513689,0.717953


In [ ]:
for col in data_embed.select_dtypes(include='float').columns:
    data_embed[col] = data_embed[col].astype('float32')

In [ ]:
embedding = data_embed

# Prepare dt_train and dt_test

In [ ]:
train_df_paper = train_df.merge(metadata, how='left', left_on='paper', right_on='paper_id') \
                   .rename(columns={'publication_year': 'paper_year', 'publication_date': 'paper_date'}) \
                   .drop(columns=['paper_id', 'content', 'doi', 'title', 'cited_by_count', 'type', 'authors', 'concepts'])

train_df_new = train_df_paper.merge(metadata, how='left', left_on='referenced_paper', right_on='paper_id') \
                   .rename(columns={'publication_year': 'ref_paper_year', 'publication_date': 'ref_paper_date'}) \
                   .drop(columns=['paper_id', 'content', 'doi', 'title', 'cited_by_count', 'type', 'authors', 'concepts'])

In [ ]:
train_df_new['paper_date'] = pd.to_datetime(train_df_new['paper_date'], format='mixed', errors='coerce')
train_df_new['ref_paper_date'] = pd.to_datetime(train_df_new['ref_paper_date'], format='mixed', errors='coerce')

In [ ]:
# Filter baris yang valid: paper_date > ref_paper_date
train_df_cleaned = train_df_new[train_df_new['paper_date'] > train_df_new['ref_paper_date']].reset_index(drop=True)

In [ ]:
dt_train = train_df_cleaned.copy()
dt_train = dt_train[['paper', 'referenced_paper', 'is_referenced']]

In [ ]:
test_df_paper = test_df.merge(metadata, how='left', left_on='paper', right_on='paper_id') \
                   .rename(columns={'publication_year': 'paper_year', 'publication_date': 'paper_date'}) \
                   .drop(columns=['paper_id', 'content', 'doi', 'title', 'cited_by_count', 'type', 'authors', 'concepts'])

test_df_new = test_df_paper.merge(metadata, how='left', left_on='referenced_paper', right_on='paper_id') \
                   .rename(columns={'publication_year': 'ref_paper_year', 'publication_date': 'ref_paper_date'}) \
                   .drop(columns=['paper_id', 'content', 'doi', 'title', 'cited_by_count', 'type', 'authors', 'concepts'])

In [ ]:
test_df_new['paper_date'] = pd.to_datetime(test_df_new['paper_date'], format='mixed', errors='coerce')
test_df_new['ref_paper_date'] = pd.to_datetime(test_df_new['ref_paper_date'], format='mixed', errors='coerce')

In [ ]:
test_df_cleaned = test_df_new[test_df_new['paper_date'] > test_df_new['ref_paper_date']].reset_index(drop=True)
test_df_0 = test_df_new[test_df_new['paper_date'] <= test_df_new['ref_paper_date']].reset_index(drop=True)

# Handle Imbalance Data

In [ ]:
dt_train

,paper,referenced_paper,is_referenced
0,p2128,p3728,0
1,p1298,p3760,0
2,p0211,p1808,0
3,p0843,p2964,0
4,p1606,p3627,0
...,...,...,...
266002,p2256,p3507,0
266003,p1032,p0730,0
266004,p0719,p2382,1
266005,p4213,p0457,0


In [ ]:
# Balancing the Imbalance Data
df_majority = dt_train[dt_train['is_referenced'] == 0]
df_minority = dt_train[dt_train['is_referenced'] == 1]
df_majority_sampled = df_majority.sample(n = 6500, random_state = 2025)
df_balanced = pd.concat([df_majority_sampled, df_minority], axis = 0).sample(frac = 1, random_state = 2025).reset_index(drop=True)

In [ ]:
df_balanced

,paper,referenced_paper,is_referenced
0,p2483,p0981,0
1,p0725,p1704,1
2,p0782,p2732,0
3,p3064,p1405,1
4,p3658,p0586,1
...,...,...,...
10592,p3921,p1024,0
10593,p3560,p0702,1
10594,p3509,p1847,0
10595,p0080,p0829,0


# Merging Train & Embedded

In [ ]:
batch_size = 5000
num_batches = (len(df_balanced) + batch_size - 1) // batch_size

merged_batches = []

embedding_cols = [col for col in embedding.columns if col != 'paper_id']

embedding_paper = embedding.rename(columns={'paper_id': 'paper'})
embedding_ref = embedding.rename(columns={'paper_id': 'referenced_paper'})
embedding_ref = embedding_ref.rename(columns={col: f"{col}_ref" for col in embedding_cols})

for i in range(num_batches):
    start = i * batch_size
    end = min((i + 1) * batch_size, len(df_balanced))

    batch = df_balanced.iloc[start:end].copy()

    # Merge embedding
    batch = batch.merge(embedding_paper, on='paper', how='left')
    batch = batch.merge(embedding_ref, on='referenced_paper', how='left')

    # Buat fitur baru
    for col in embedding_cols:
        ref_col = f"{col}_ref"
        batch[f"{col}_abs_diff"] = (batch[col] - batch[ref_col]).abs()
        batch[f"{col}_mul"] = batch[col] * batch[ref_col]

    merged_batches.append(batch)
    print(f"✅ Batch {i+1}/{num_batches} selesai digabung & dihitung.")

# Gabung semua batch
train_df_with_embed = pd.concat(merged_batches, ignore_index=True)

✅ Batch 1/3 selesai digabung & dihitung.
✅ Batch 2/3 selesai digabung & dihitung.
✅ Batch 3/3 selesai digabung & dihitung.


In [ ]:
train_df_with_embed

,paper,referenced_paper,is_referenced,dim_0,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6,...,mtdt_763_abs_diff,mtdt_763_mul,mtdt_764_abs_diff,mtdt_764_mul,mtdt_765_abs_diff,mtdt_765_mul,mtdt_766_abs_diff,mtdt_766_mul,mtdt_767_abs_diff,mtdt_767_mul
0,p2483,p0981,0,-0.216859,0.467223,-0.809789,-0.034501,-0.247385,0.353072,0.601202,...,0.382217,-0.030982,0.664769,0.615720,1.276122,-0.373337,0.972628,-0.223302,0.133626,0.024774
1,p0725,p1704,1,-0.285272,0.571589,0.877238,-0.182401,0.297220,-0.856047,0.312085,...,0.044452,0.024835,0.300502,-0.022573,0.030306,0.015065,0.420410,0.857826,0.184417,1.397279
2,p0782,p2732,0,-0.300498,0.542462,-0.038007,0.172063,-0.082823,-0.189893,1.093386,...,0.159195,0.006307,1.092071,-0.031410,0.740996,0.125553,1.379470,-0.397638,1.055579,0.147415
3,p3064,p1405,1,-0.182774,0.585294,-0.077243,0.590000,-0.663280,0.238440,1.134478,...,0.106910,0.019728,0.750646,-0.069042,0.367496,0.652594,0.325389,0.000663,0.389469,0.927421
4,p3658,p0586,1,-0.819823,-0.249222,-0.032340,-0.164345,-0.367130,-0.228473,1.458627,...,0.293247,0.014278,0.249919,0.090094,0.036655,0.218296,0.005848,0.000493,0.169910,0.768086
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10592,p3921,p1024,0,0.358312,0.315352,-0.748817,0.715001,-0.558609,0.414044,0.549428,...,0.458871,0.028271,0.141301,0.027548,0.635540,-0.099873,0.309719,-0.023383,0.384165,0.802947
10593,p3560,p0702,1,-0.374393,0.867002,-0.417435,0.470116,-0.285632,-0.120061,0.670968,...,1.029470,-0.254251,0.133620,0.015410,0.325031,0.117755,0.462013,0.137234,0.540410,1.466624
10594,p3509,p1847,0,-0.195052,0.496944,0.581423,-0.196783,-0.316882,-0.612991,1.147740,...,0.182327,-0.002791,0.178938,0.050734,0.469848,0.705918,0.429587,0.214241,1.370306,-0.360105
10595,p0080,p0829,0,0.073932,0.379445,-0.236112,0.018178,-0.769028,0.346248,-1.124123,...,1.061936,-0.277933,0.003037,0.875896,1.521550,-0.520755,0.511617,-0.016631,1.681394,0.308604


# Modeling

In [ ]:
X = train_df_with_embed.drop(columns=['paper', 'referenced_paper', 'is_referenced'])
y = train_df_with_embed['is_referenced']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=2025, stratify = y
)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((8477, 6144), (2120, 6144), (8477,), (2120,))

In [ ]:
class LSTMClassifier:
    def __init__(self, input_shape, epochs=5, batch_size=32, patience=3):
        self.input_shape = input_shape
        self.epochs = epochs
        self.batch_size = batch_size
        self.patience = patience
        self.model = None

    def fit(self, X, y):
        X = X.reshape((-1, 1, self.input_shape))

        self.model = Sequential([
            Masking(mask_value=0., input_shape=(1, self.input_shape)),
            LSTM(16),
            Dense(1, activation='sigmoid')
        ])
        self.model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

        early_stop = EarlyStopping(
            monitor='loss',
            patience=self.patience,
            restore_best_weights=True,
            verbose=1
        )

        self.model.fit(
            X, y,
            epochs=self.epochs,
            batch_size=self.batch_size,
            callbacks=[early_stop],
            verbose=1
        )

    def predict(self, X):
        X = X.reshape((-1, 1, self.input_shape))
        proba = self.model.predict(X, verbose=0)
        return (proba > 0.5).astype(int).flatten()

    def predict_proba(self, X):
        X = X.reshape((-1, 1, self.input_shape))
        return self.model.predict(X, verbose=0).flatten()

In [ ]:
# TSSE-BIM Boosting-style ensemble using custom base model (e.g., LSTM)
class TSSEBIMBoost:
    def __init__(self, n_estimators=5, base_model_class=None, input_shape=None, **base_model_kwargs):
        self.n_estimators = n_estimators
        self.base_model_class = base_model_class
        self.input_shape = input_shape
        self.base_model_kwargs = base_model_kwargs
        self.classifiers = []
        self.weights = []

    def _bin_samples(self, probs, bins):
        return np.digitize(probs, bins) - 1

    def _assign_weight(self, fp, fn, fp_max, fn_max):
        return (1 - fp / (fp_max + 1e-10)) * (1 - fn / (fn_max + 1e-10))

    def fit(self, X, y):
        X, y = shuffle(X, y, random_state=42)
        pos_idx = np.where(y == 1)[0]
        neg_idx = np.where(y == 0)[0]
        k = len(pos_idx)
        n_bins = k if k > 0 else 1
        bins = np.linspace(0, 1, n_bins + 1)

        pred_probs_all = np.zeros(len(y))

        for t in range(self.n_estimators):
            if t > 0:
                pred_probs_all = np.zeros(len(y))
                total_weight = sum(self.weights)
                for clf, w in zip(self.classifiers, self.weights):
                    pred_probs_all += w * clf.predict_proba(X)
                pred_probs_all /= total_weight
            else:
                pred_probs_all[:] = 0.5

            neg_probs = pred_probs_all[neg_idx]
            bin_assignments = self._bin_samples(neg_probs, bins)
            neg_sampled_idx = []

            for i in range(n_bins):
                bin_indices = neg_idx[bin_assignments == i]
                if len(bin_indices) > 0:
                    neg_sampled = np.random.choice(bin_indices, 1, replace=False)
                    neg_sampled_idx.extend(neg_sampled)

            train_idx = np.concatenate([pos_idx, neg_sampled_idx])
            clf = self.base_model_class(self.input_shape)
            clf.fit(X[train_idx], y[train_idx])
            y_pred = clf.predict(X)

            tn, fp, fn, tp = confusion_matrix(y, y_pred).ravel()
            fp_max = max([fp] + [confusion_matrix(y, c.predict(X)).ravel()[1] for c in self.classifiers]) if self.classifiers else fp
            fn_max = max([fn] + [confusion_matrix(y, c.predict(X)).ravel()[2] for c in self.classifiers]) if self.classifiers else fn

            weight = self._assign_weight(fp, fn, fp_max, fn_max)
            self.classifiers.append(clf)
            self.weights.append(weight)

    def predict(self, X):
        total_weight = sum(self.weights)
        preds = sum(w * clf.predict(X) for clf, w in zip(self.classifiers, self.weights))
        return (preds / total_weight) > 0.5

    def predict_proba(self, X):
        total_weight = sum(self.weights)
        probs = sum(w * clf.predict_proba(X) for clf, w in zip(self.classifiers, self.weights))
        return probs / total_weight

In [ ]:
X_train = np.array(X)
y_train = np.array(y)

boost_model = TSSEBIMBoost(
    n_estimators=5,
    base_model_class=LSTMClassifier,
    input_shape=X.shape[1],
    base_model_kwargs={
        'epochs': 20,
        'batch_size': 32,
        'patience': 3
    }
)

# Latih model
boost_model.fit(X_train, y_train)

Epoch 1/5
129/129 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.9607 - loss: 0.2303
Epoch 2/5
129/129 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 1.0000 - loss: 0.0907
Epoch 3/5
129/129 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 1.0000 - loss: 0.0525
Epoch 4/5
129/129 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - accuracy: 1.0000 - loss: 0.0344
Epoch 5/5
129/129 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.9996 - loss: 0.0258
Restoring model weights from the end of the best epoch: 5.
Epoch 1/5
129/129 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy: 0.9996 - loss: 0.1311
Epoch 2/5
129/129 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - accuracy: 0.9996 - loss: 0.0534
Epoch 3/5
129/129 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.9993 - loss: 0.0329
Epoch 4/5
129/129 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.9993 - loss: 0.0229
Epoch 5/5
129/129 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.9999 - loss: 0.0147
Restoring model weights from the end of the best epoch: 5.
Epoch 1/5
129/129 ━━━━

In [ ]:
# Label Predict
y_pred = boost_model.predict(X_test)

y_proba = boost_model.predict_proba(X_test)

print(classification_report(y, y_pred))

AttributeError: 'DataFrame' object has no attribute 'reshape'

In [ ]:
# XGBoost
xgb_model = XGBClassifier(
    tree_method='gpu_hist',
    predictor='gpu_predictor',
    gpu_id=0,
    n_estimators=100,
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=2025,
)

xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)

print("⚡ XGBoost")
print(classification_report(y_test, y_pred_xgb))

⚡ XGBoost
              precision    recall  f1-score   support

           0       0.89      0.92      0.90      1300
           1       0.87      0.81      0.84       820

    accuracy                           0.88      2120
   macro avg       0.88      0.87      0.87      2120
weighted avg       0.88      0.88      0.88      2120



In [ ]:
mcc_xgb = matthews_corrcoef(y_test, y_pred_xgb)

In [ ]:
print("Accuracy:")
print(f"XGB : {accuracy_score(y_test, y_pred_xgb):.4f}")

print("")

print("📈 Matthews Correlation Coefficient")
print(f"XGB : {mcc_xgb:.4f}")

Accuracy:
XGB : 0.8807

📈 Matthews Correlation Coefficient
XGB : 0.7464


# With LSTM

In [ ]:
n_samples_train = X_train.shape[0]
n_samples_test = X_test.shape[0]
timesteps = 3
features = X_train.shape[1] // timesteps

In [ ]:
X_train_lstm = X_train.values.reshape((n_samples_train, timesteps, features))
X_test_lstm = X_test.values.reshape((n_samples_test, timesteps, features))

In [ ]:
# LSTM Model
model = models.Sequential([
    layers.LSTM(64, return_sequences=True, input_shape=(timesteps, features)),
    layers.Dropout(0.6),
    layers.LSTM(32, return_sequences=False),
    layers.Dropout(0.5),
    layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])


In [ ]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Earlystopping
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True,
    mode='min'
)

In [ ]:
history = model.fit(
    X_train_lstm, y_train,
    epochs=50,
    batch_size=32,
    validation_data=(X_test_lstm, y_test),
    verbose=0,
    callbacks=[TqdmCallback(verbose=1), early_stop]
)

0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

In [ ]:
import joblib
joblib.dump(xgb_model, 'model_xgb_1.pkl')

['model_xgb_1.pkl']

In [ ]:
model.save('model_lstm.keras')
model.save_weights('lstm_weights_base.weights.h5')

# Finetune XGB

In [ ]:
X_tr, X_val, y_tr, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42, stratify=y_train
)

def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 5),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 5),
        'use_label_encoder': False,
        'eval_metric': 'logloss',
        'random_state': 2025,
        'tree_method': 'gpu_hist',
        'predictor': 'gpu_predictor',
        'gpu_id': 0,
    }

    model = XGBClassifier(**params)
    model.fit(X_tr, y_tr)
    preds = model.predict(X_val)
    f1 = f1_score(y_val, preds, average='macro')
    return f1

In [ ]:
# Buat dan jalankan studi Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

[I 2025-04-24 00:44:23,620] A new study created in memory with name: no-name-1cf1c5b8-95df-4a49-9e9a-1c1905b01597
[I 2025-04-24 00:45:09,465] Trial 0 finished with value: 0.8578267500341141 and parameters: {'n_estimators': 110, 'max_depth': 9, 'learning_rate': 0.046625566159516435, 'subsample': 0.7172159673513914, 'colsample_bytree': 0.9309305461266135, 'gamma': 4.221026470577533, 'reg_alpha': 4.446368231954924, 'reg_lambda': 2.23864468148598}. Best is trial 0 with value: 0.8578267500341141.
[I 2025-04-24 00:45:44,851] Trial 1 finished with value: 0.8051022753390025 and parameters: {'n_estimators': 74, 'max_depth': 8, 'learning_rate': 0.012538981461005239, 'subsample': 0.5507791639587509, 'colsample_bytree': 0.8052991397535261, 'gamma': 3.038567376502248, 'reg_alpha': 4.363953544060398, 'reg_lambda': 0.5630683341598353}. Best is trial 0 with value: 0.8578267500341141.
[I 2025-04-24 00:46:12,113] Trial 2 finished with value: 0.8547019002886254 and parameters: {'n_estimators': 181, 'max_

In [ ]:
# Tampilkan hasil terbaik
print("✅ Best trial:")
print(study.best_trial)

✅ Best trial:
FrozenTrial(number=47, state=1, values=[0.877924134456201], datetime_start=datetime.datetime(2025, 4, 24, 1, 9, 8, 889132), datetime_complete=datetime.datetime(2025, 4, 24, 1, 9, 31, 985494), params={'n_estimators': 276, 'max_depth': 3, 'learning_rate': 0.1757199345744328, 'subsample': 0.9696270609661942, 'colsample_bytree': 0.5246063036711618, 'gamma': 0.5624570862750969, 'reg_alpha': 0.724073392826929, 'reg_lambda': 0.24576713709293393}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'n_estimators': IntDistribution(high=300, log=False, low=50, step=1), 'max_depth': IntDistribution(high=15, log=False, low=3, step=1), 'learning_rate': FloatDistribution(high=0.3, log=False, low=0.01, step=None), 'subsample': FloatDistribution(high=1.0, log=False, low=0.5, step=None), 'colsample_bytree': FloatDistribution(high=1.0, log=False, low=0.5, step=None), 'gamma': FloatDistribution(high=5.0, log=False, low=0.0, step=None), 'reg_alpha': FloatDistribution(high=

In [ ]:
best_params = study.best_params
best_params.update({
    'use_label_encoder': False,
    'eval_metric': 'logloss',
    'random_state': 2025,
    'tree_method': 'gpu_hist',
    'predictor': 'gpu_predictor',
    'gpu_id': 0,
})

In [ ]:
best_model = XGBClassifier(**best_params)
best_model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.5246063036711618, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='logloss', feature_types=None,
              gamma=0.5624570862750969, gpu_id=0, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.1757199345744328, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=276, n_jobs=None,
              num_parallel_tree=None, ...)

In [ ]:
# Prediksi dan evaluasi
y_pred = best_model.predict(X_test)

from sklearn.metrics import classification_report
print("\n⚡ Final XGBoost (GPU) Evaluation:")
print(classification_report(y_test, y_pred))


⚡ Final XGBoost (GPU) Evaluation:
              precision    recall  f1-score   support

           0       0.90      0.93      0.91      1300
           1       0.88      0.84      0.86       820

    accuracy                           0.89      2120
   macro avg       0.89      0.88      0.89      2120
weighted avg       0.89      0.89      0.89      2120



In [ ]:
# Save Best Params
import joblib
joblib.dump(best_model, 'best_xgb_2.pkl')

['best_xgb_2.pkl']

# LSTM FINETUNE

In [ ]:
from optuna.integration import TFKerasPruningCallback

def create_lstm_model(trial):
    model = models.Sequential()

    # Param tuning
    units1 = trial.suggest_int('units1', 32, 128)
    dropout1 = trial.suggest_float('dropout1', 0.3, 0.7)
    units2 = trial.suggest_int('units2', 16, 64)
    dropout2 = trial.suggest_float('dropout2', 0.3, 0.7)
    dense_units = trial.suggest_int('dense_units', 32, 128)
    dense_dropout = trial.suggest_float('dense_dropout', 0.3, 0.7)
    l2_reg = trial.suggest_float('l2_reg', 1e-4, 1e-2, log=True)
    lr = trial.suggest_float('lr', 1e-4, 1e-2, log=True)

    model.add(layers.LSTM(units1, return_sequences=True, input_shape=(timesteps, features)))
    model.add(layers.Dropout(dropout1))
    model.add(layers.LSTM(units2, return_sequences=False))
    model.add(layers.Dropout(dropout2))
    model.add(layers.Dense(dense_units, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l2_reg)))
    model.add(layers.Dropout(dense_dropout))
    model.add(layers.Dense(1, activation='sigmoid'))

    optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    return model

def objective(trial):
    model = create_lstm_model(trial)

    early_stop = EarlyStopping(
        monitor='val_loss',
        patience=3,
        restore_best_weights=True
    )

    model.fit(
        X_train_lstm, y_train,
        validation_data=(X_test_lstm, y_test),
        epochs=30,
        batch_size=32,
        verbose=0,
        callbacks=[early_stop]
    )

    # Predict & compute f1_score
    y_pred_prob = model.predict(X_test_lstm)
    y_pred = (y_pred_prob > 0.5).astype(int)
    f1 = f1_score(y_test, y_pred, average='binary')
    return f1  # Karena kita pengen maximize f1

In [ ]:
# Jalankan Optuna
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=30)

[I 2025-04-24 01:12:35,693] A new study created in memory with name: no-name-14794d7f-7840-462f-8e40-32fcdc8127e8


67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step


[I 2025-04-24 01:13:19,982] Trial 0 finished with value: 0.8690176322418136 and parameters: {'units1': 95, 'dropout1': 0.610513312880824, 'units2': 60, 'dropout2': 0.5395985692679546, 'dense_units': 96, 'dense_dropout': 0.3531041483293317, 'l2_reg': 0.0003290165341517008, 'lr': 0.00030263723632762226}. Best is trial 0 with value: 0.8690176322418136.


67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step


[I 2025-04-24 01:13:51,516] Trial 1 finished with value: 0.8716094032549728 and parameters: {'units1': 67, 'dropout1': 0.6317110749587823, 'units2': 28, 'dropout2': 0.6630067013167328, 'dense_units': 58, 'dense_dropout': 0.42036158506865906, 'l2_reg': 0.0007238895094497908, 'lr': 0.00012687345707440136}. Best is trial 0 with value: 0.8690176322418136.


...

[I 2025-04-24 01:25:39,317] Trial 27 finished with value: 0.8419737663960025 and parameters: {'units1': 121, 'dropout1': 0.344906583672296, 'units2': 42, 'dropout2': 0.5306867764373757, 'dense_units': 128, 'dense_dropout': 0.6622781609725253, 'l2_reg': 0.006298642842433657, 'lr': 0.009993267388731934}. Best is trial 12 with value: 0.8238276299112801.


67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step


[I 2025-04-24 01:26:08,335] Trial 28 finished with value: 0.8473748473748474 and parameters: {'units1': 109, 'dropout1': 0.39649912821106514, 'units2': 60, 'dropout2': 0.6332901409370442, 'dense_units': 39, 'dense_dropout': 0.47938457846472543, 'l2_reg': 0.00470902249079871, 'lr': 0.006916417955561353}. Best is trial 12 with value: 0.8238276299112801.


67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step


[I 2025-04-24 01:26:55,422] Trial 29 finished with value: 0.8587223587223587 and parameters: {'units1': 97, 'dropout1': 0.47946828069627173, 'units2': 64, 'dropout2': 0.5461022085159606, 'dense_units': 92, 'dense_dropout': 0.35879395135103953, 'l2_reg': 0.0021444153185149755, 'lr': 0.00454801053250985}. Best is trial 12 with value: 0.8238276299112801.


In [ ]:
# Print best result
print("✅ Best trial:")
print(study.best_trial)

✅ Best trial:
FrozenTrial(number=12, state=1, values=[0.8238276299112801], datetime_start=datetime.datetime(2025, 4, 24, 1, 18, 18, 849440), datetime_complete=datetime.datetime(2025, 4, 24, 1, 18, 47, 600487), params={'units1': 127, 'dropout1': 0.3026058364786072, 'units2': 50, 'dropout2': 0.5794781100349495, 'dense_units': 32, 'dense_dropout': 0.6935093531119139, 'l2_reg': 0.009889196751726791, 'lr': 0.009743170562809675}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'units1': IntDistribution(high=128, log=False, low=32, step=1), 'dropout1': FloatDistribution(high=0.7, log=False, low=0.3, step=None), 'units2': IntDistribution(high=64, log=False, low=16, step=1), 'dropout2': FloatDistribution(high=0.7, log=False, low=0.3, step=None), 'dense_units': IntDistribution(high=128, log=False, low=32, step=1), 'dense_dropout': FloatDistribution(high=0.7, log=False, low=0.3, step=None), 'l2_reg': FloatDistribution(high=0.01, log=True, low=0.0001, step=None), 'lr': Float

In [ ]:
# Train ulang pakai best params
final_model = create_lstm_model(study.best_trial)
final_model.fit(
    X_train_lstm, y_train,
    validation_data=(X_test_lstm, y_test),
    epochs=30,
    batch_size=32,
    verbose=1,
    callbacks=[EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)]
)

Epoch 1/30
265/265 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.6015 - loss: 0.7621 - val_accuracy: 0.6623 - val_loss: 0.5202
Epoch 2/30
265/265 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.7153 - loss: 0.5724 - val_accuracy: 0.7976 - val_loss: 0.4977
Epoch 3/30
265/265 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.7066 - loss: 0.5842 - val_accuracy: 0.8505 - val_loss: 0.4429
Epoch 4/30
265/265 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.7494 - loss: 0.5382 - val_accuracy: 0.8792 - val_loss: 0.3838
Epoch 5/30
265/265 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.7877 - loss: 0.4908 - val_accuracy: 0.8642 - val_loss: 0.3892
Epoch 6/30
265/265 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.7866 - loss: 0.4993 - val_accuracy: 0.8665 - val_loss: 0.3859
Epoch 7/30
265/265 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.7919 - loss: 0.4897 - val_accuracy: 0.8736 - val_loss: 0.3746
Epoch 8/30
265/265 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.7921 - loss: 0.5039 - val_accuracy: 

In [ ]:
# Evaluasi akhir
y_pred_final = (final_model.predict(X_test_lstm) > 0.5).astype(int)
print("⚡ Final Tuned LSTM")
print(classification_report(y_test, y_pred_final))

67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step
⚡ Final Tuned LSTM
              precision    recall  f1-score   support

           0       0.90      0.89      0.90      1300
           1       0.83      0.84      0.84       820

    accuracy                           0.87      2120
   macro avg       0.87      0.87      0.87      2120
weighted avg       0.87      0.87      0.87      2120



In [ ]:
final_model.save('model_lstm_tuning.keras')
final_model.save_weights('lstm_weights_tune.weights.h5')